<a href="https://colab.research.google.com/github/SalimBobo/SalimBoBoMohirDev/blob/main/Pneumonia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Funksiyalar va kutubxonalarni import qilish
import torch
import fastai
from fastai.vision.all import *
from ipywidgets import widgets
import os
from PIL import Image
import numpy as np
import pandas as pd
from google.colab import drive
from zipfile import ZipFile

# Torch va fastai versiyalarini chiqarish
print(torch.__version__)
print(fastai.__version__)

# Google Drive'ga ulashish
drive.mount('/content/drive')

# Ma'lumotlarni yuklash va ko'rish
train_zip_path = '/content/drive/MyDrive/train.zip'
test_zip_path = '/content/drive/MyDrive/test.zip'

with ZipFile(train_zip_path, 'r') as zip:
  zip.extractall('/content/train')
  print("Train ma'lumotlari ekstrakt qilindi")

with ZipFile(test_zip_path, 'r') as zip:
  zip.extractall('/content/test')
  print("Test ma'lumotlari ekstrakt qilindi")

# Train data
path = Path('/content/train')

# Ma'lumotlarni yuklash
db = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224)
)

# Datalar yuklash
dls = db.dataloaders(path)

# Modelni yaratish
learn = vision_learner(dls, resnet152, metrics=accuracy)

# Modelni bo'lish
learn.fine_tune(10)

# Modelni qayta o'rnatish
learn.unfreeze()
learn.fit_one_cycle(4, lr_max=slice(1e-6,1e-4))

# Interpretatsiya
interp = ClassificationInterpretation.from_learner(learn)

# Matrisini namoyish etish
interp.plot_confusion_matrix()

# Test ma'lumotlari uchun path
path = Path('/content/test')
images = [os.path.join(path, f) for f in os.listdir(path)]

# Predictionlar ro'yxati
predictions = []
for i in images:
  pred, pred_id, _ = learn.predict(i)
  predictions.append(pred)

# DataFrame larini yaratish
df1 = pd.DataFrame(images)
df2 = pd.DataFrame(predictions)
df = pd.concat([df1, df2], axis=1)

# CSV fayliga saqlash
df.to_csv('/content/submission.csv')

# Exploarary data analysis
# Ma'lumotlar statistik tahlili
print(df.describe())

# Ma'lumotlar vizualizatsiyasi
import seaborn as sns
import matplotlib.pyplot as plt

# Categorical o'lchovlar uchun histogram
sns.countplot(data=df, x='Category')
plt.show()

# Correlation lar orasidagi xususiyatlarni aniqlash
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()
